In [1]:
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import Dataset, DataLoader, random_split

# 导入训练图片的路径
trans_compose = transforms.Compose([
    # 转换为torch数据类型
    transforms.ToTensor(),
    # 调整图片大小
    transforms.Resize((100, 100))
])

ds = ImageFolder(r'C:\Users\seven\Downloads\imgs\raw-img', trans_compose)

# 划分训练和测试的数据
train, test = random_split(ds, (25000, 1179))
# 设定每次读取的数量
data_loader = DataLoader(dataset=train, batch_size=3000, shuffle=True, num_workers=4)
data_loader2 = DataLoader(dataset=test, batch_size=3000, shuffle=True, num_workers=4)

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.L1 = nn.Conv2d(3, 30, 3, 2, 1)
        self.B1 = nn.BatchNorm2d(30)
        self.L2 = nn.Conv2d(30, 60, 5, 2, 1)
        self.B2 = nn.BatchNorm2d(60)
        self.L = nn.Linear(34560, 10)

    def forward(self, x):
        g1 = self.L1(x)
        g1 = self.B1(g1)
        f1 = torch.relu(g1)

        g2 = self.L2(f1)
        g2 = self.B2(g2)
        f2 = torch.relu(g2)

        f2 = torch.flatten(f2, start_dim=1)
        g = self.L(f2)

        return g

# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = MyNet().to(device)

lossf = nn.CrossEntropyLoss()     # loss函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)  # SGD函数
metricsf = torchmetrics.Accuracy(task='multiclass', num_classes=10).to(device)  # torchmetrics函数



Using device: cuda


In [2]:
# 训练模型
for i in range(100):
    model.train()
    for batchX, batchY in data_loader:
        batchX, batchY = batchX.to(device), batchY.to(device)

        optimizer.zero_grad()
        score = model(batchX)
        loss = lossf(score, batchY)
        loss.backward() #求导
        optimizer.step() #梯度下降

        mse = metricsf(score, batchY)
    print(f"Epoch {i+1} - Loss: {loss.item()}, Accuracy: {metricsf.compute().item()}")
    metricsf.reset()

torch.save(model.state_dict(), 'model_gpu.pth')



Epoch 1 - Loss: 17.59947395324707, Accuracy: 0.16380000114440918
Epoch 2 - Loss: 4.942524433135986, Accuracy: 0.17107999324798584
Epoch 3 - Loss: 4.6387939453125, Accuracy: 0.21904000639915466
Epoch 4 - Loss: 3.71565318107605, Accuracy: 0.24447999894618988
Epoch 5 - Loss: 4.501423358917236, Accuracy: 0.2685199975967407
Epoch 6 - Loss: 3.410369873046875, Accuracy: 0.2858799993991852
Epoch 7 - Loss: 2.161977529525757, Accuracy: 0.31619998812675476
Epoch 8 - Loss: 2.2524447441101074, Accuracy: 0.32552000880241394
Epoch 9 - Loss: 1.653964877128601, Accuracy: 0.39684000611305237
Epoch 10 - Loss: 2.0232677459716797, Accuracy: 0.40112000703811646


KeyboardInterrupt: 

In [ ]:
# 测试模型
model.eval()
with torch.no_grad():
    for batchX, batchY in data_loader2:
        batchX, batchY = batchX.to(device), batchY.to(device)
        score = model(batchX)
        mse = metricsf(score, batchY)

print(f"Test Accuracy: {metricsf.compute().item()}")
